# Baixa Dados

Aqui serão baixados todos os dados do formulário desejado.

Os arquivos serão salvos na pasta data deste repositório

In [1]:
from dotenv import load_dotenv

load_dotenv()
import requests
import pandas as pd
import geopandas as gpd
import os
import json
from normaliza_iniciativas.normalizer import clear_string

In [2]:
KOBO_API_KEY = os.getenv("KOBO_API_KEY")
KOBO_USERNAME = os.getenv("KOBO_USERNAME")
KOBO_PASSWORD = os.getenv("KOBO_PASSWORD")
KOBO_URL = os.getenv("KOBO_URL")
FROM_UID = "atUcxaKhdkAJsHhiPzXJ2E"

data_folder = "data"
os.makedirs(data_folder, exist_ok=True)

kobo_endpoints = {
    "token": f"{KOBO_URL}/token/",
    "assets_json": f"{KOBO_URL}/api/v2/assets.json",
}
auth = (KOBO_USERNAME, KOBO_PASSWORD)
opts = {
    "format": "json",
}

In [3]:
result = requests.post(kobo_endpoints["token"], auth=auth, params=opts)
API_TOKEN = result.json()["token"]
headers = {"Authorization": f"Token {API_TOKEN}"}

Aqui pegamos as informções de todos os formulários que estão no kobotoolbox, e salvamos em um arquivo json.

In [4]:
result_assets = requests.get(kobo_endpoints["assets_json"], headers=headers)
result_assets = result_assets.json()
print(f"Quantidade de resultados {result_assets['count']}")
print(f'Next: {result_assets["next"]}')
with open(os.path.join('.',data_folder, "kobo_assets.json"), "w") as f:
    json.dump(result_assets, f, indent=2, ensure_ascii=False)

results = result_assets["results"]


Quantidade de resultados 19
Next: None


Baixando todos os dados do formulário que desejamos

In [5]:
form_to_download = next(filter(lambda x: x["uid"] == FROM_UID, results))
form_name = clear_string(form_to_download["name"])
print(f"Baixando {form_name}")
# * Salvando arquivo de metadados do formulário
filename_form_metadata = f"{form_name}_metadata.json"
with open(os.path.join(".", data_folder, filename_form_metadata), "w") as f:
    json.dump(form_to_download, f, indent=2, ensure_ascii=False)
# * Baixando o arquivo xls do formulário
print("Baixando arquivo xls do formulário")
download_url = next(filter(lambda x: x["format"] == "xls", form_to_download["downloads"]))
form_configs_filename = f"{form_name}_configs.xlsx"
form_configs = requests.get(download_url["url"], headers=headers)
with open(os.path.join(".", data_folder, form_configs_filename), "wb") as f:
    f.write(form_configs.content)
# * Baixando os dados do formulário
print("Baixando dados do formulário")
url_data = form_to_download["data"]
data_filename = f"{form_name}_data.json"
data = requests.get(url_data, headers=headers)
data = data.json()
with open(os.path.join(".", data_folder, data_filename), "w") as f:
    json.dump(data, f, indent=2, ensure_ascii=False)

print("Dados Baixados com sucesso!")

Baixando cadastro_de_iniciativas_mapa_das_periferias
Baixando arquivo xls do formulário
Baixando dados do formulário
Dados Baixados com sucesso!
